In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
import seaborn as sns
  

In [2]:
data_dir = 'C:/Users/hs'

In [3]:
#Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize([224,224]),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize([256,256]),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

#pass transform here-in
train_data = datasets.ImageFolder(data_dir + '/trainS', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/testC', transform=test_transforms)
test_dataA = datasets.ImageFolder(data_dir + '/testA', transform=test_transforms)

#data loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True,num_workers=4)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True,num_workers=4)
testloaderA = torch.utils.data.DataLoader(test_dataA, batch_size=10, shuffle=True,num_workers=2)

print("Classes: ")
class_names = train_data.classes
print(class_names)

Classes: 
['Infected', 'Normal']


In [15]:
vgg16=models.vgg16(pretrained=True)

In [16]:
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
print(features)
print(num_features)

[Linear(in_features=25088, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False)]
4096


In [17]:
features.extend([nn.Linear(num_features, 2)])
# features.extend([nn.ReLU(inplace=True)])
# features.extend([nn.Linear(280, 2)])


vgg16.classifier = nn.Sequential(*features)


In [ ]:
# i=0
# for name, param in vgg16.named_parameters():
#     if i<16
#         param.requires_grad=False 

#     i=i+1

In [ ]:
# print(vgg16)
# i=0
# for name,param in vgg16.features.named_parameters():
#     if param.requires_grad:
#         print(vgg16.features[i])
#         print('-------------------------------------')
#     i=i+1
# for name, param in vgg16.named_parameters():
#     if param.requires_grad:
#         print(name)

In [8]:
def accuracyE(model,Aloader):
    correct = 0
    total = 0

    with torch.no_grad():
        for data in Aloader:
            images, labels = data
#             actualclasses.append(labels.tolist())
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
#             predictedclasses.append(predicted.tolist())
    #         print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100*correct/total
#     print('Accuracy of the network on the 10000 test images: %d %%' % (
#         100 * correct / total))

In [10]:
Epochs = 4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

In [13]:
# comloss=[]
# accuracy=[]
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
#     print(type(trainloader))

    pbar = tqdm(enumerate(trainloader))
    print(pbar)
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = vgg16(inputs)#----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
#         print(loss.item())
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.item()))
#     acc=accuracyE(vgg16,testloaderA)
#     accuracy.append(acc)
#     comloss.append(running_loss)    
    torch.save(resnet18.state_dict(), 'vgg16_entire.pth')

print('Finished Training')

KeyboardInterrupt: 